In [17]:
filename = '../data/TIEGCM_CCMC/s010.nc'

In [18]:
import pandas as pd
import numpy as np
from tiegcm.tiegcm import TIEGCM, Point3D, Point4D, Slice_key4D, ColumnSlice4D, ColumnSlice3D
from tiegcm.util import average_longitude
from tiegcm.util import *
from scipy.spatial import kdtree
from scipy.interpolate import griddata, LinearNDInterpolator
import scipy

from netCDF4 import Dataset

from plotly.offline import plot, iplot
import plotly.graph_objs as go
from plotly.graph_objs import Layout
import plotly
plotly.offline.init_notebook_mode(connected=True)

import matplotlib.pyplot as plt

In [19]:
tiegcm = TIEGCM(filename)  # Imports data file

initializing tiegcm with ../data/TIEGCM_CCMC/s010.nc


In [4]:







time = 12
z_max = tiegcm.z.max()

z_test= 1.1*z_max
print 'max height is at',z_max*1e-5,'km \n','above that we can extrapolate to get', z_test*1e-5,'km'

p4 = Point4D(time, z_test, 20.5, .5)

print 'We can store all data into a Point4d:', p4

max height is at 588.9554400000001 km 
above that we can extrapolate to get 647.8509840000002 km
We can store all data into a Point4d: Point4D(time=12, height=64785098.400000006, latitude=20.5, longitude=0.5)


In [5]:
column_slicer = tiegcm.get_column_slicer_4D(p4)
z_column, lat_column, lon_column = tiegcm.get_3D_column(column_slicer)

time_index = column_slicer.time.start

In [6]:
z = tiegcm.z[time_index, -1, :, :]
print z
lat = tiegcm.lat_[-1]
print lat
lon = tiegcm.lon_[-1]
print lon

[[54234064. 54234064. 54234064. ... 54234064. 54234064. 54234064.]
 [53924390. 53885820. 53849852. ... 54008216. 53965292. 53924390.]
 [53483596. 53412910. 53348852. ... 53643636. 53560624. 53483596.]
 ...
 [54846004. 54794576. 54751124. ... 54970490. 54904852. 54846004.]
 [55474770. 55475492. 55479090. ... 55481996. 55476936. 55474770.]
 [55858800. 55858800. 55858800. ... 55858800. 55858800. 55858800.]]
[[-90.  -90.  -90.  ... -90.  -90.  -90. ]
 [-87.5 -87.5 -87.5 ... -87.5 -87.5 -87.5]
 [-82.5 -82.5 -82.5 ... -82.5 -82.5 -82.5]
 ...
 [ 82.5  82.5  82.5 ...  82.5  82.5  82.5]
 [ 87.5  87.5  87.5 ...  87.5  87.5  87.5]
 [ 90.   90.   90.  ...  90.   90.   90. ]]
[[-180. -175. -170. ...  170.  175.  180.]
 [-180. -175. -170. ...  170.  175.  180.]
 [-180. -175. -170. ...  170.  175.  180.]
 ...
 [-180. -175. -170. ...  170.  175.  180.]
 [-180. -175. -170. ...  170.  175.  180.]
 [-180. -175. -170. ...  170.  175.  180.]]


In [7]:
outer_points = np.array(zip(z.ravel()/z.max(), lat.ravel(), lon.ravel()))
tree = scipy.spatial.KDTree(outer_points)

z_test = 1.1*tiegcm.z[time_index, -1, :, :].max() # high altitude test

p4 = Point4D(time, z_test, 20.5, .5)
p = Point3D(*p4[1:])

distances, vertices = tree.query(p, p = 1, k = 3)

coord_indices = np.array(zip(*np.unravel_index(vertices, z.shape)))

lat_indices, lon_indices = coord_indices[:,0], coord_indices[:,1]

lnd_lat = LinearNDInterpolator(tree.data[vertices][:,1:], tiegcm.lat_[-1][lat_indices, lon_indices])
lnd_lon = LinearNDInterpolator(tree.data[vertices][:,1:], tiegcm.lon_[-1][lat_indices, lon_indices])
lnd_z = LinearNDInterpolator(tree.data[vertices][:,1:], tiegcm.z[time_index,-1, lat_indices, lon_indices])
assert (float(lnd_lat(p[1:])) == p.latitude)
assert (float(lnd_lon(p[1:])) == p.longitude)

zz_top = z_column[:,-1,:,:]
assert zz_top.min() <= lnd_z(p[1:]) <= zz_top.max()


pd.Series(column_slicer._asdict()).to_frame().T

tiegcm.z[time_index,-1,lat_indices, lon_indices]

tiegcm.z[time_index,-1,:,:].min(), tiegcm.z[time_index,-1,:,:].max()

tree.data[vertices][:,1:]

z_column.shape, lat_column.shape, lon_column.shape

lat_column[-1], lon_column[-1]

tiegcm.z[time_index, -1, :, :].max()

height_index = -1

norm = tiegcm.z[time_index, height_index, :, :].max()


In [8]:
data = [go.Surface( z=tiegcm.z[time_index, height_index, :, :]/norm,
                    x=tiegcm.lon,
                    y=tiegcm.lat,
                    opacity = .9,
                   name = 'max_height'
                    ),
        go.Surface( z=z_column[0, height_index, :, :]/norm,
                    x=lon_column[height_index],
                    y = lat_column[height_index],
                    opacity = 1,
                   name = 'column_height',
                    ),
        go.Scatter3d(x = [p.longitude], y =[ p.latitude],z = [p.height/norm], name = 'query', mode = 'markers'),
        go.Scatter3d(x = [lon[c[0], c[1]] for c in coord_indices], 
                     y = [lat[c[0], c[1]] for c in coord_indices], 
                     z = tiegcm.z[time_index,height_index, lat_indices, lon_indices]/norm, mode = 'markers')
]

iplot(data)

In [9]:
lat = tiegcm.lat
lon = tiegcm.lon
time = 3.5


Z = tiegcm.rootgrp.variables['Z'].__array__()
ZG = tiegcm.rootgrp.variables['ZG'].__array__()
DEN = tiegcm.rootgrp.variables['DEN'].__array__()

/Users/zwaldron/anaconda3/envs/kamodo/lib/python2.7/site-packages/ipykernel_launcher.py:6: UserWarning:

cannot be safely cast to variable data type

/Users/zwaldron/anaconda3/envs/kamodo/lib/python2.7/site-packages/ipykernel_launcher.py:8: UserWarning:

cannot be safely cast to variable data type



# Mass Density

In [10]:
z_max = tiegcm.z.max()
z_test = 1.1*z_max

print z_max
print z_test

point = Point3D(z_test, -20.5, .5)
density_tot = tiegcm.time_interpolate(point, 'DEN', 3.5)

print point

58895544.0
64785098.400000006
Point3D(height=64785098.400000006, latitude=-20.5, longitude=0.5)


In [11]:
tiegcm.mass_density(point,time)

molecular_mass

NameError: name 'molecular_mass' is not defined